In [1]:
%load_ext autoreload
%autoreload 2

import scipy.io.wavfile as wav
import pyaudio
import wave
import sys
import pyaudio
import scipy
from scipy import signal

import numpy as np
import matplotlib.pylab as plt
from scipy.stats import norm
import matplotlib.mlab as mlab
from numpy import random
#from scipy.fftpack import fft, ifft

from playsound import *
from savetimeseries import *
from combinesignalnoise import *
from combinesignalsignal import *
from plottimeseries import *
from plotsignalnoisetimeseries import *
from plotsignalsignaltimeseries import *
from plotsignalsignalnoisetimeseries import *
from plothistogram import *
from plotpowerspectrum import *

%matplotlib inline

# Different types of searches

# cross-correlation for stochastic signals

In [2]:
# white gwb in white noise in two detectors
A = 0.1
datafile_det1 = 'whitegwb_in_noise_det1_0.1.txt'
datafile_det2 = 'whitegwb_in_noise_det2_0.1.txt'

noisefile_det1 = 'whitenoise_det1.txt'
noisefile_det2 = 'whitenoise_det2.txt'

signalfile = 'whitenoise.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# extract signal and noise components for comparison
ts = np.loadtxt(signalfile)
s = A*ts[:,1]

ts = np.loadtxt(noisefile_det1)
n1 = ts[:,1]

ts = np.loadtxt(noisefile_det2)
n2 = ts[:,1]

# cross-correlate
N = len(t)
C11 = np.sum(d1**2)/N
C22 = np.sum(d2**2)/N
C12 = np.sum(d1*d2)/N

N11 = np.sum(n1**2)/N
N22 = np.sum(n2**2)/N
gwb = np.sum(s**2)/N

# estimates and SNR calculation
Sh = C12
Sn1 = C11-C12
Sn2 = C22-C12
SNR2 = N*Sh**2/(Sn1*Sn2)
SNR = np.sqrt(SNR2)

print gwb, N11, N22
print Sh, Sn1, Sn2, SNR
print ""

# compare with expected values
deltaT = t[1]-t[0]
pow_meas = 2.*deltaT*Sh
pow_exp  = 2.*deltaT*A**2
percent_error = 100*np.abs(pow_meas-pow_exp)/pow_exp
pow_variance = (2.*deltaT)**2 * Sn1*Sn2/N

print "expected  =", pow_exp
print "estimated =", pow_meas, "+/-", np.sqrt(pow_variance)
print "range = [", pow_meas-np.sqrt(pow_variance), pow_meas+np.sqrt(pow_variance), "]"
print "percent error =", percent_error
print "SNR =", pow_meas/np.sqrt(pow_variance)

0.00996817957318 1.00276135362 0.998918258357
0.00965249459122 1.0033269109 0.998775998188 2.89271430072

expected  = 8.88898765542e-07
estimated = 8.58009052653e-07 +/- 2.96610367792e-07
range = [ 5.61398684862e-07 1.15461942045e-06 ]
percent error = 3.4750540878
SNR = 2.89271430072


In [3]:
# confusion-limited BNS chirps in white noise in two detectors (N=10, A=0.03)
A = 0.03
datafile_det1 = 'chirpBNS10_in_noise_det1_0.03.txt'
datafile_det2 = 'chirpBNS10_in_noise_det2_0.03.txt'

noisefile_det1 = 'whitenoise_det1.txt'
noisefile_det2 = 'whitenoise_det2.txt'

signalfile = 'chirpBNS10.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# extract signal and noise components for comparison
ts = np.loadtxt(signalfile)
s = A*ts[:,1]

ts = np.loadtxt(noisefile_det1)
n1 = ts[:,1]

ts = np.loadtxt(noisefile_det2)
n2 = ts[:,1]

# cross-correlate
N = len(t)
C11 = np.sum(d1**2)/N
C22 = np.sum(d2**2)/N
C12 = np.sum(d1*d2)/N

N11 = np.sum(n1**2)/N
N22 = np.sum(n2**2)/N
gwb = np.sum(s**2)/N

# estimates and SNR calculation
Sh = C12
Sn1 = C11-C12
Sn2 = C22-C12
SNR2 = N*Sh**2/(Sn1*Sn2)
SNR = np.sqrt(SNR2)

print gwb, N11, N22
print Sh, Sn1, Sn2, SNR

0.00509319840351 1.00276135362 0.998918258357
0.00505340755936 1.00387462465 0.998228284438 1.51443593618


# optimal filtering for cross-correlation search

In [4]:
# white gwb in white noise in two detectors
A = 0.1
datafile_det1 = 'whitegwb_in_noise_det1_0.1.txt'
datafile_det2 = 'whitegwb_in_noise_det2_0.1.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# fourier transform data
deltaT = t[1]-t[0]
d1tilde = deltaT * np.fft.fft(d1)
d2tilde = deltaT * np.fft.fft(d2)

# construct optimal filter
N = len(t)
deltaF = 1/t[-1]
var1 = np.sum(d1**2)/N
var2 = np.sum(d2**2)/N
P1 = 2.*deltaT*var1*np.ones(N) # one-sided
P2 = 2.*deltaT*var2*np.ones(N)
H  = np.ones(N) # unit norm
Q = H/(P1*P2)
norm = 1./((t[-1]/2.)*np.sum(deltaF * pow(H,2)/(P1*P2)))

# do cross-correlation integral
temp = np.sum(deltaF * Q * d1tilde * np.conj(d2tilde))
Y = np.real(temp) # take real part to avoid imaginary round-off
Y = norm*Y

# SNR calculation
sigma2 = 0.5*norm
SNR = Y/np.sqrt(sigma2)

# compare measured with expected amplitude
pow_meas = Y
pow_exp  = 2.*deltaT*A**2
percent_error = 100*np.abs(pow_meas-pow_exp)/pow_exp

print "expected  =", pow_exp
print "estimated =", pow_meas, "+/-", np.sqrt(sigma2)
print "range = [", pow_meas-np.sqrt(sigma2), pow_meas+np.sqrt(sigma2), "]"
print "percent error =", percent_error
print "SNR =", SNR

expected  = 8.88898765542e-07
estimated = 8.58018586193e-07 +/- 2.99470405285e-07
range = [ 5.58548180908e-07 1.15748899148e-06 ]
percent error = 3.47398157649
SNR = 2.86511979498


In [5]:
# confusion-limited BNS chirps in white noise in two detectors using optimal filter
A = 0.03
datafile_det1 = 'chirpBNS10_in_noise_det1_0.03.txt'
datafile_det2 = 'chirpBNS10_in_noise_det2_0.03.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# fourier transform data
deltaT = t[1]-t[0]
d1tilde = deltaT * np.fft.fft(d1)
d2tilde = deltaT * np.fft.fft(d2)

# calculate discrete frequencies 
N = len(t)
deltaF = 1.0/(N*deltaT)
fNyq = 1.0/(2*deltaT)
if ( np.mod(N,2)== 0 ):
    numFreqs = N/2 - 1
else:
    numFreqs = (N-1)/2

# discrete positive frequencies
fp = np.linspace(deltaF, numFreqs*deltaF, numFreqs)

# discrete frequencies (including zero and negative frequencies)
if ( np.mod(N,2)== 0 ):
    a = np.hstack( (np.array([0.]), fp) )
    b = np.hstack( (np.array([-fNyq]), np.flipud(-fp)) )
    f = np.hstack( (a, b) )
else:
    f = np.hstack( (np.hstack(((np.array([0.]),fp))), np.flipud(-fp)) )

# expected signal PSD for chirps
f0 = 100.
H = 1.e-4*np.ones(N)
ndx = np.where(np.abs(f)>=f0)[0]
H[ndx] = 1./pow(np.abs(f[ndx])/f0,7./3.)

# construct optimal filter
var1 = np.sum(d1**2)/N
var2 = np.sum(d2**2)/N
P1 = 2.*deltaT*var1*np.ones(len(d1tilde)) # one-sided
P2 = 2.*deltaT*var2*np.ones(len(d1tilde))
Q = H/(P1*P2)
norm = 1./((t[-1]/2.)*np.sum(deltaF * pow(H,2)/(P1*P2)))

# do cross-correlation integral
temp = np.sum(deltaF * Q * d1tilde * np.conj(d2tilde))
Y = np.real(temp) # take real part to avoid imaginary round-off
Y = norm*Y

# SNR calculation
sigma2 = 0.5*norm
SNR = Y/np.sqrt(sigma2)

# compare measured with expected amplitude
max_power_single_burst = 8.e-3
N_bursts = 10
pow_meas = Y
pow_exp  = max_power_single_burst * N_bursts * A**2
percent_error = 100*np.abs(pow_meas-pow_exp)/pow_exp

print "expected  =", pow_exp
print "estimated =", pow_meas, "+/-", np.sqrt(sigma2)
print "range = [", pow_meas-np.sqrt(sigma2), pow_meas+np.sqrt(sigma2), "]"
print "percent error =", percent_error
print "SNR =", SNR

expected  = 7.2e-05
estimated = 7.39625154589e-05 +/- 6.06835971232e-06
range = [ 6.78941557466e-05 8.00308751712e-05 ]
percent error = 2.72571591511
SNR = 12.188222018


In [6]:
# white gwb + confusion-limited BNS chirps in white noise in two detectors (10 chirps, A=0.1, A=0.03)
datafile_det1 = 'whitegwb_chirpBNS10_in_noise_det1.txt'
datafile_det2 = 'whitegwb_chirpBNS10_in_noise_det2.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# fourier transform data
deltaT = t[1]-t[0]
d1tilde = deltaT * np.fft.fft(d1)
d2tilde = deltaT * np.fft.fft(d2)

# construct optimal filter
N = len(t)
deltaF = 1/t[-1]
var1 = np.sum(d1**2)/N
var2 = np.sum(d2**2)/N
P1 = 2.*deltaT*var1*np.ones(N) # one-sided
P2 = 2.*deltaT*var2*np.ones(N)
H  = np.ones(N) # unit norm
Q = H/(P1*P2)
norm = 1./((t[-1]/2.)*np.sum(deltaF * pow(H,2)/(P1*P2)))

# do cross-correlation integral
temp = np.sum(deltaF * Q * d1tilde * np.conj(d2tilde))
Y = np.real(temp) # take real part to avoid imaginary round-off
Y = norm*Y

# SNR calculation
sigma2 = 0.5*norm
SNR = Y/np.sqrt(sigma2)

# compare measured with expected amplitude
A = 0.1
pow_meas = Y
pow_exp  = 2.*deltaT*A**2
percent_error = 100*np.abs(pow_meas-pow_exp)/pow_exp

print "white noise component:"
print "expected  =", pow_exp
print "estimated =", pow_meas, "+/-", np.sqrt(sigma2)
print "range = [", pow_meas-np.sqrt(sigma2), pow_meas+np.sqrt(sigma2), "]"
print "percent error =", percent_error
print "SNR =", SNR
print " "

#####################################################
# target confusion-limited BNS chirp component
A = 0.03

# calculate discrete frequencies 
N = len(t)
deltaF = 1.0/(N*deltaT)
fNyq = 1.0/(2*deltaT)
if ( np.mod(N,2)== 0 ):
    numFreqs = N/2 - 1
else:
    numFreqs = (N-1)/2

# discrete positive frequencies
fp = np.linspace(deltaF, numFreqs*deltaF, numFreqs)

# discrete frequencies (including zero and negative frequencies)
if ( np.mod(N,2)== 0 ):
    a = np.hstack( (np.array([0.]), fp) )
    b = np.hstack( (np.array([-fNyq]), np.flipud(-fp)) )
    f = np.hstack( (a, b) )
else:
    f = np.hstack( (np.hstack(((np.array([0.]),fp))), np.flipud(-fp)) )

# expected signal PSD for chirps
f0 = 100.
H = 1.e-4*np.ones(N)
ndx = np.where(np.abs(f)>=f0)[0]
H[ndx] = 1./pow(np.abs(f[ndx])/f0,7./3.)

# construct optimal filter
var1 = np.sum(d1**2)/N
var2 = np.sum(d2**2)/N
P1 = 2.*deltaT*var1*np.ones(len(d1tilde)) # one-sided
P2 = 2.*deltaT*var2*np.ones(len(d1tilde))
Q = H/(P1*P2)
norm = 1./((t[-1]/2.)*np.sum(deltaF * pow(H,2)/(P1*P2)))

# do cross-correlation integral
temp = np.sum(deltaF * Q * d1tilde * np.conj(d2tilde))
Y = np.real(temp) # take real part to avoid imaginary round-off
Y = norm*Y

# SNR calculation
sigma2 = 0.5*norm
SNR = Y/np.sqrt(sigma2)

# compare measured with expected amplitude
max_power_single_burst = 8.e-3
N_bursts = 10
pow_meas = Y
pow_exp  = max_power_single_burst * N_bursts * A**2
percent_error = 100*np.abs(pow_meas-pow_exp)/pow_exp

print "BNS confusion component:"
print "expected  =", pow_exp
print "estimated =", pow_meas, "+/-", np.sqrt(sigma2)
print "range = [", pow_meas-np.sqrt(sigma2), pow_meas+np.sqrt(sigma2), "]"
print "percent error =", percent_error
print "SNR =", SNR

white noise component:
expected  = 8.88898765542e-07
estimated = 1.32052101524e-06 +/- 3.01011349884e-07
range = [ 1.01950966536e-06 1.62153236513e-06 ]
percent error = 48.5569635638
SNR = 4.3869475877
 
BNS confusion component:
expected  = 7.2e-05
estimated = 7.69902906242e-05 +/- 6.12747575354e-06
range = [ 7.08628148707e-05 8.31177663778e-05 ]
percent error = 6.93095920032
SNR = 12.5647646308


In [7]:
# popcorn BBH chirps, confusion BNS background in white noise in two detectors using optimal filter
A = 0.003
datafile_det1 = 'chirpBNS50BBH10_in_noise_det1_0.003.txt'
datafile_det2 = 'chirpBNS50BBH10_in_noise_det2_0.003.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# fourier transform data
deltaT = t[1]-t[0]
d1tilde = deltaT * np.fft.fft(d1)
d2tilde = deltaT * np.fft.fft(d2)

# calculate discrete frequencies 
N = len(t)
deltaF = 1.0/(N*deltaT)
fNyq = 1.0/(2*deltaT)
if ( np.mod(N,2)== 0 ):
    numFreqs = N/2 - 1
else:
    numFreqs = (N-1)/2

# discrete positive frequencies
fp = np.linspace(deltaF, numFreqs*deltaF, numFreqs)

# discrete frequencies (including zero and negative frequencies)
if ( np.mod(N,2)== 0 ):
    a = np.hstack( (np.array([0.]), fp) )
    b = np.hstack( (np.array([-fNyq]), np.flipud(-fp)) )
    f = np.hstack( (a, b) )
else:
    f = np.hstack( (np.hstack(((np.array([0.]),fp))), np.flipud(-fp)) )

# expected signal PSD for chirps
f0 = 100.
H = 1.e-4*np.ones(N)
ndx = np.where(np.abs(f)>=f0)[0]
H[ndx] = 1./pow(np.abs(f[ndx])/f0,7./3.)

# construct optimal filter
var1 = np.sum(d1**2)/N
var2 = np.sum(d2**2)/N
P1 = 2.*deltaT*var1*np.ones(len(d1tilde)) # one-sided
P2 = 2.*deltaT*var2*np.ones(len(d1tilde))
Q = H/(P1*P2)
norm = 1./((t[-1]/2.)*np.sum(deltaF * pow(H,2)/(P1*P2)))

# do cross-correlation integral
temp = np.sum(deltaF * Q * d1tilde * np.conj(d2tilde))
Y = np.real(temp) # take real part to avoid imaginary round-off
Y = norm*Y

# SNR calculation
sigma2 = 0.5*norm
SNR = Y/np.sqrt(sigma2)

# compare measured with expected amplitude
max_power_single_burst = 1.
N_bursts = 10
pow_meas = Y
pow_exp  = max_power_single_burst * N_bursts * A**2
percent_error = 100*np.abs(pow_meas-pow_exp)/pow_exp

print "expected  =", pow_exp
print "estimated =", pow_meas, "+/-", np.sqrt(sigma2)
print "range = [", pow_meas-np.sqrt(sigma2), pow_meas+np.sqrt(sigma2), "]"
print "percent error =", percent_error
print "SNR =", SNR

expected  = 9e-05
estimated = 3.38500058176e-05 +/- 3.82772766141e-06
range = [ 3.00222781562e-05 3.7677733479e-05 ]
percent error = 62.3888824249
SNR = 8.84336839291


## Component separation method

In [8]:
# component separation method

# white gwb + confusion-limited BNS chirps in white noise in two detectors (10 chirps, A=0.1, A=0.03)
datafile_det1 = 'whitegwb_chirpBNS10_in_noise_det1.txt'
datafile_det2 = 'whitegwb_chirpBNS10_in_noise_det2.txt'

# extract data
ts = np.loadtxt(datafile_det1)
t = ts[:,0]
d1 = ts[:,1]

ts = np.loadtxt(datafile_det2)
t = ts[:,0]
d2 = ts[:,1]

# fourier transform data
deltaT = t[1]-t[0]
d1tilde = deltaT * np.fft.fft(d1)
d2tilde = deltaT * np.fft.fft(d2)
D = (2./t[-1]) * d1tilde * np.conj(d2tilde)

# calculate discrete frequencies 
N = len(t)
deltaF = 1.0/(N*deltaT)
fNyq = 1.0/(2*deltaT)
if ( np.mod(N,2)== 0 ):
    numFreqs = N/2 - 1
else:
    numFreqs = (N-1)/2

# discrete positive frequencies
fp = np.linspace(deltaF, numFreqs*deltaF, numFreqs)

# discrete frequencies (including zero and negative frequencies)
if ( np.mod(N,2)== 0 ):
    a = np.hstack( (np.array([0.]), fp) )
    b = np.hstack( (np.array([-fNyq]), np.flipud(-fp)) )
    f = np.hstack( (a, b) )
else:
    f = np.hstack( (np.hstack(((np.array([0.]),fp))), np.flipud(-fp)) )

# construct noise covariance matrix
N = len(t)
deltaF = 1/t[-1]
var1 = np.sum(d1**2)/N
var2 = np.sum(d2**2)/N
P1 = 2.*deltaT*var1*np.ones(N) # one-sided
P2 = 2.*deltaT*var2*np.ones(N)
N12 = P1 * P2
invN = 1./N12

# construct optimal filters for components 1 and 2
H1 = np.ones(N) # unit norm
M1 = H1 # overlap function = 1 for this example

# expected signal PSD for chirps
f0 = 100.
H2 = 1.e-4*np.ones(N)
ndx = np.where(np.abs(f)>=f0)[0]
H2[ndx] = 1./pow(np.abs(f[ndx])/f0,7./3.)
M2 = H2 # overlap function = 1 for this example

# do cross-correlation integral for component #1
temp = np.sum(deltaF * M1 * invN * D)
X1 = np.real(temp) # take real part to avoid imaginary round-off

# do cross-correlation integral for component #2
temp = np.sum(deltaF * M2 * invN * D)
X2 = np.real(temp) # take real part to avoid imaginary round-off
#print "X =", X1, X2

# calculate the fisher matrix
F11 = np.real(np.sum(deltaF * M1 * M1 * invN))
F12 = np.real(np.sum(deltaF * M1 * M2 * invN))
F21 = np.real(np.sum(deltaF * M2 * M1 * invN))
F22 = np.real(np.sum(deltaF * M2 * M2 * invN))
#print "F =", F11, F12, F21, F22

# calculate inverse fisher matrix
detF = F11*F22-F12*F21
              
invF11 =  (1./detF)*F22
invF12 = -(1./detF)*F12
invF21 = -(1./detF)*F21
invF22 =  (1./detF)*F11
#print "invF =", invF11, invF12, invF21, invF22

# calculate ML estimator of the two components
Y1 = invF11*X1 + invF12*X2
Y2 = invF21*X1 + invF22*X2
#print "Y =", Y1, Y2

# uncertainties
sigma1 = np.sqrt(invF11)
sigma2 = np.sqrt(invF22)

# compare ML values with expected values of the power
# white noise
A1 = 0.1
expY1  = 2.*deltaT*A1**2
percent_error1 = 100*np.abs(Y1-expY1)/expY1
SNR1 = Y1/sigma1

# BNS confusion
A2 = 0.03
max_power_single_burst = 8.e-3
N_bursts = 10
expY2  = max_power_single_burst * N_bursts * A2**2
percent_error2 = 100*np.abs(Y2-expY2)/expY2
SNR2 = Y2/sigma2

print "white noise component:"
print "expected  =", expY1
print "estimated =", Y1, "+/-", sigma1
print "range = [", Y1-sigma1, Y1+sigma1, "]"
print "percent error =", percent_error1
print "SNR =", SNR1
print " "
print "BNS confusion component:"
print "expected  =", expY2
print "estimated =", Y2, "+/-", sigma2 
print "range = [", Y2-sigma2, Y2+sigma2, "]"
print "percent error =", percent_error2
print "SNR =", SNR2

white noise component:
expected  = 8.88898765542e-07
estimated = 8.24045049557e-07 +/- 6.07605988101e-07
range = [ 2.16439061456e-07 1.43165103766e-06 ]
percent error = 7.29596198118
SNR = 1.35621614285
 
BNS confusion component:
expected  = 7.2e-05
estimated = 7.4721490204e-05 +/- 1.23685378552e-05
range = [ 6.23529523488e-05 8.70900280593e-05 ]
percent error = 3.77984750561
SNR = 6.04125492266


In [9]:
# try preconditioning, gives the same result

# precondition fisher matrix
B11 = F11/np.sqrt(F11*F11)
B12 = F12/np.sqrt(F11*F22)
B21 = F21/np.sqrt(F22*F11)
B22 = F22/np.sqrt(F22*F22)
print "B =", B11, B12, B21, B22
print ""

# calculate inverse of preconditioned fisher matrix
detB = B11*B22-B12*B21
              
invB11 =  (1./detB)*B22
invB12 = -(1./detB)*B12
invB21 = -(1./detB)*B21
invB22 =  (1./detB)*B11
#print "invB =", invB11, invB12, invB21, invB22

# precondition X
Z1 = X1/np.sqrt(F11)
Z2 = X2/np.sqrt(F22)

# calculate ML estimator of the two components
YY1 = invB11*Z1 + invB12*Z2
YY2 = invB21*Z1 + invB22*Z2

Y1 = YY1/np.sqrt(F11)
Y2 = YY2/np.sqrt(F22)

# uncertainties
sigma1 = np.sqrt(invB11/F11)
sigma2 = np.sqrt(invB22/F22)

################################################
# compare ML values with expected values of the power
# white noise
A1 = 0.1
expY1  = 2.*deltaT*A1**2
percent_error1 = 100*np.abs(Y1-expY1)/expY1
SNR1 = Y1/sigma1

# BNS confusion
A2 = 0.03
max_power_single_burst = 8.e-3
N_bursts = 10
expY2  = max_power_single_burst * N_bursts * A2**2
percent_error2 = 100*np.abs(Y2-expY2)/expY2
SNR2 = Y2/sigma2

print "white noise component:"
print "expected  =", expY1
print "estimated =", Y1, "+/-", sigma1
print "range = [", Y1-sigma1, Y1+sigma1, "]"
print "percent error =", percent_error1
print "SNR =", SNR1
print " "
print "BNS confusion component:"
print "expected  =", expY2
print "estimated =", Y2, "+/-", sigma2 
print "range = [", Y2-sigma2, Y2+sigma2, "]"
print "percent error =", percent_error2
print "SNR =", SNR2

B = 1.0 0.135253658706 0.135253658706 1.0

white noise component:
expected  = 8.88898765542e-07
estimated = 8.24045049557e-07 +/- 6.07605988101e-07
range = [ 2.16439061456e-07 1.43165103766e-06 ]
percent error = 7.29596198118
SNR = 1.35621614285
 
BNS confusion component:
expected  = 7.2e-05
estimated = 7.4721490204e-05 +/- 1.23685378552e-05
range = [ 6.23529523488e-05 8.70900280593e-05 ]
percent error = 3.77984750561
SNR = 6.04125492266
